In [8]:
import os
import sys


working_dir = '/Users/amitshendge/Documents/Real-Estate-Bot'

In [11]:
from PyPDF2 import PdfReader
pdf_path = os.path.join(working_dir, 'skyslope_forms/FORMS/NVAR Standard Forms Library - Affiliate Attorney Members/Addendum Lease - K1384.pdf')
tagged_form_path = os.path.join(working_dir, 'skyslope_forms/FORMS/tagged_NVAR')

reader = PdfReader(os.path.join(working_dir, 'skyslope_forms/FORMS/NVAR Standard Forms Library - Affiliate Attorney Members/Addendum Lease - K1384.pdf'))
fields = reader.get_form_text_fields()
print(fields)  # Prints all field names in the form


{}


In [15]:
import fitz  # PyMuPDF

def read_pdf_to_text(pdf_path):
    """
    Reads and extracts text from a PDF file.
    """
    doc = fitz.open(pdf_path)
    all_text = ""

    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text()  # Extract text from the page
        all_text += f"Page {page_num + 1}:\n{text}\n\n"

    return all_text


# Input PDF file
input_pdf = pdf_path

# Extract text
extracted_text = read_pdf_to_text(input_pdf)

# Print or save the extracted text
print(extracted_text)

# Optionally save the text to a file
# with open("output_text.txt", "w", encoding="utf-8") as f:
#     f.write(extracted_text)
#     print("Text extracted and saved to output_text.txt")


Page 1:
NVAR – K1384 – rev. 01/19 
ADDENDUM - LEASE 
 
This Addendum is made on   
 
 
, to a residential lease (“Lease”) ratified on  
 
 
, 
between   
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (“Tenant”) and  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (“Landlord”) for 
the lease of Premises:   
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
. 
The parties agree that this Lease is modified as follows: 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
This Addendum shall not alter, modify, or change in any other respect this Lease, and except as 
modified herein, all of the terms and provisions of this Lease are expressly ratified and confirmed 
and shall remain in full force and effect. 
 
LANDLORD: 
 
 
 
 
 
 
 
 
 
TENANT: 
 
 
 
 
/ 
 
 
 
 
 
 
 
 
 
 
 
 
 
/ 
 
 
 
 
 
 
 
 
 
Date 
 
 
 
Signature  
 
 
 
 
 
Date 
 
 
 
Signature 
 
 
 
 
/ 
 
 
 
 
 
 
 
 
 
 
 
 
 
/ 
 
 
 
 
 
 
 
 
 
Date 
 
 
 
Signature  
 
 
 
 
 
Date 
 
 
 
Signature 
 
 
 
 
/ 
 
 
 


In [55]:
import fitz  # PyMuPDF

def find_text_coordinates(pdf_path, search_text):
    """
    Find the coordinates of specific text in a PDF.
    """
    doc = fitz.open(pdf_path)
    results = []

    for page_num in range(len(doc)):
        page = doc[page_num]
        
        # Search for the text on the page
        text_instances = page.search_for(search_text)

        # Collect results with page number and coordinates
        for inst in text_instances:
            results.append({
                "page": page_num + 1,
                "coordinates": inst  # Coordinates as [x0, y0, x1, y1]
            })

    return results


# Input PDF file
input_pdf = pdf_path
search_text = "made on"  # Replace with the text you're searching for

# Find text coordinates
coordinates = find_text_coordinates(input_pdf, search_text)

# Print results
if coordinates:
    for result in coordinates:
        print(f"Page {result['page']}: Coordinates {result['coordinates']}")
else:
    print(f"'{search_text}' not found in the PDF.")


Page 1: Coordinates Rect(163.6800079345703, 98.98399353027344, 204.6120147705078, 114.99199676513672)


In [65]:
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from io import BytesIO

# Function to get the size of the first page of an existing PDF
def get_pdf_page_size(pdf_path):
    reader = PdfReader(pdf_path)
    page = reader.pages[0]
    width = float(page.mediabox[2])  # Upper-right x-coordinate
    height = float(page.mediabox[3])  # Upper-right y-coordinate
    return width, height

# Function to add a text field
def add_text_field(c, name, tooltip, x, y, width=50, height=20):
    c.drawString(x - 50, y + 5, f"{name.capitalize()}:")
    c.acroForm.textfield(
        name=name,
        tooltip=tooltip,
        x=x,
        y=y,
        width=width,
        height=height,
        borderStyle='solid',
        forceBorder=True,
    )

# Function to add a checkbox field
def add_checkbox_field(c, name, tooltip, x, y, size=20):
    c.drawString(x - 50, y + 5, f"{name.capitalize()}:")
    c.acroForm.checkbox(
        name=name,
        tooltip=tooltip,
        x=x,
        y=y,
        size=size,
        borderStyle='solid',
        forceBorder=True,
    )

# Create a new layer with form fields using reportlab
def create_form_fields_layer(input_pdf_path):
    # Get the page size from the input PDF
    page_width, page_height = get_pdf_page_size(input_pdf_path)
    packet = BytesIO()
    c = canvas.Canvas(packet, pagesize=(page_width, page_height))

    # Add form fields as needed
    add_text_field(c, name="name", tooltip="Enter your name", x=163.6800079345703, y=page_height-98)
    # add_checkbox_field(c, name="subscribe", tooltip="Subscribe to newsletter", x=150, y=page_height - 150)

    c.save()
    packet.seek(0)
    return packet

# Merge the form fields layer onto the existing PDF
def add_fields_to_existing_pdf(input_pdf_path, output_pdf_path):
    # Read the existing PDF
    pdf_reader = PdfReader(input_pdf_path)
    pdf_writer = PdfWriter()

    # Generate the form fields layer
    fields_pdf = create_form_fields_layer(input_pdf_path)

    # Add the form fields to the first page of the existing PDF
    fields_reader = PdfReader(fields_pdf)
    for i, page in enumerate(pdf_reader.pages):
        if i == 0:  # Overlay fields only on the first page
            page.merge_page(fields_reader.pages[0])
        pdf_writer.add_page(page)

    # Write the updated PDF to a new file
    with open(output_pdf_path, 'wb') as output_file:
        pdf_writer.write(output_file)

# Paths to input and output PDFs
input_pdf_path = pdf_path
output_pdf_path = 'output_with_form_fields.pdf'

# Add form fields to the existing PDF
add_fields_to_existing_pdf(input_pdf_path, output_pdf_path)
print(f"Form fields added to: {output_pdf_path}")


Form fields added to: output_with_form_fields.pdf
